In [1]:
import numpy as np
from numba import njit
import random
from datetime import datetime

In [2]:
class Environment:

    def __init__(self):
        self.board = np.zeros((4, 4)).astype(np.uint8)
        i, j = self.random_choice()
        self.board[i][j] = Environment.generate_new()
        self.cul_reward = 0

    def random_choice(self):
        rows, cols = np.where(self.board == 0)
        if len(rows) == 0:
            return None, None
        idx = random.randint(0, len(rows) - 1)
        return rows[idx].item(), cols[idx].item()

    def apply_each_row_core(self, row_step, board):
        # print(row_step([4, 4, 0, 0]))
        new_rows = []
        total_rewards = 0
        for row in board:
            new_row, reward = row_step(row)
            new_rows.append(new_row)
            total_rewards += reward
        return np.array(new_rows), total_rewards

    def apply_to_each_row(self, row_step):
        return self.apply_each_row_core(row_step, self.board)

    def apply_to_each_col(self, col_step):
        new_board_T, total_rewards = self.apply_each_row_core(col_step, self.board.T)
        return new_board_T.T, total_rewards

    def generate_new():
        return 2 if random.random() < 0.9 else 4

    def reverse_squeeze(row):
        new_row, reward = squeeze(row[::-1])
        return new_row[::-1], reward

    def move_left(self):
        new_board, reward = self.apply_to_each_row(squeeze)
        self.board = new_board
        self.cul_reward += reward
        return new_board, reward
        # new_board = np.apply_along_axis(squeeze, axis=1, arr=self.board)
        

    def move_right(self):
        new_board, reward = self.apply_to_each_row(Environment.reverse_squeeze)
        self.board = new_board
        self.cul_reward += reward
        return new_board, reward
        
        # return np.apply_along_axis(reverse_squeeze, axis=1, arr=self.board)

    def move_up(self):
        # return np.apply_along_axis(squeeze, axis=0, arr=self.board)
        new_board, reward = self.apply_to_each_col(squeeze)
        self.board = new_board
        self.cul_reward += reward
        return new_board, reward

    def move_down(self):
        new_board, reward = self.apply_to_each_col(Environment.reverse_squeeze)
        self.board = new_board
        self.cul_reward += reward
        return new_board, reward
        # return np.apply_along_axis(reverse_squeeze, axis=0, arr=self.board)

    def step(self, action):
        old_board = self.board
        new_board, reward = None, None
        if action == 0:
            new_board, reward = self.move_left()
        elif action == 1:
            new_board, reward = self.move_right()
        elif action == 2:
            new_board, reward = self.move_up()
        elif action == 3:
            new_board, reward = self.move_down()
        else:
            print("wrong action")

        is_done = False
        if np.any(old_board != new_board):
            i, j = self.random_choice()
            self.board[i][j] = Environment.generate_new()
            is_done = self.check_done()
        
        # print(self.board, reward)
        return self.board, reward, is_done


    def check_done(self):
        if np.any(self.board == 0):
            # print("0")
            # print(self.board == 0)
            return False
        for i in range(4):
            for j in range(3):
                if self.board[i][j] == self.board[i][j + 1]:
                    # print("1", i, j)
                    return False
        for i in range(3):
            for j in range(4):
                if self.board[i][j] == self.board[i+1][j]:
                    # print("2", i, j)
                    return False
        return True

    def get_state(self):
        return torch.tensor(self.board.reshape(-1).astype(np.float32))


In [ ]:
env = Environment()
done = False
score = 0
while not done:
    action, exp_reward, pairs = get_max_action_and_rewards(env.board, 0, 3)
    print(action)
    new_board, reward, done = env.step(action)
    score += reward
    print(new_board)

In [25]:
a = env.board.copy()

In [26]:
a.astype(np.int16)

array([[ 0,  0,  0,  0],
       [ 0,  0,  2,  2],
       [ 0,  0,  4, 32],
       [ 8, 16,  4,  2]], dtype=int16)

In [10]:
print(datetime.now())
r = get_max_action_and_rewards(env.board, 0, 3)
print(datetime.now())

2026-02-17 08:40:06.298537
2026-02-17 08:40:07.892965


In [ ]:
boarn

In [74]:
r

(1,
 12.599188503803887,
 [(0, 12.599188503803886),
  (1, 12.599188503803887),
  (2, 12.324235254541376),
  (3, 12.324235254541378)])

In [4]:
@njit
def random_choice(board):
    rows, cols = np.where(board == 0)
    if len(rows) == 0:
        return None, None
    idx = random.randint(0, len(rows) - 1)
    return rows[idx].item(), cols[idx].item()


@njit
def squeeze(row):
    reward = 0.0
    if np.sum(row != 0) == 0:
        return row, reward
        
    new_row = np.zeros(4, dtype=row.dtype)
    set_idx = 0
    start = find_next_not_zero(row, 0)
    while start != -1 and start < len(row):
        next_idx = find_next_not_zero(row, start + 1)
        
        if next_idx == -1:
            break
        if row[start] == row[next_idx]:
            new_row[set_idx] = row[start] * 2
            reward += new_row[set_idx]
            start = find_next_not_zero(row, next_idx + 1)
            set_idx += 1
        else:
            new_row[set_idx] = row[start]
            start = find_next_not_zero(row, next_idx)
            set_idx += 1
    if start != -1 and start < len(row):
        new_row[set_idx] = row[start]
    return new_row, float(reward)


@njit
def find_next_not_zero(row, idx):
    if idx >= len(row):
        return -1
    if row[idx] != 0:
        return idx
    for i in range(idx + 1, len(row)):
        if row[i] != 0:
            return i
    return -1


@njit
def apply_each_row_core(row_step, board):
    # Pre-allocate a 2D array with the same shape and type as the input board
    new_board = np.zeros_like(board)
    total_rewards = 0.0
    
    # Iterate by index to avoid dynamically building Python lists
    for i in range(board.shape[0]):
        new_row, reward = row_step(board[i])
        new_board[i] = new_row
        total_rewards += reward
        
    return new_board, total_rewards


@njit
def apply_to_each_row(row_step, board):
    return apply_each_row_core(row_step, board)


@njit
def apply_to_each_col(col_step, board):
    new_board_T, total_rewards = apply_each_row_core(col_step, board.T)
    return new_board_T.T, total_rewards


def generate_new():
    return 2 if random.random() < 0.9 else 4


@njit
def reverse_squeeze(row):
    new_row, reward = squeeze(row[::-1])
    return new_row[::-1], reward


@njit
def move_left(board):
    new_board, reward = apply_to_each_row(squeeze, board)
    return new_board, reward
    

@njit
def move_right(board):
    new_board, reward = apply_to_each_row(reverse_squeeze, board)
    return new_board, reward


@njit
def move_up(board):
    new_board, reward = apply_to_each_col(squeeze, board)
    return new_board, reward


@njit
def move_down(board):
    new_board, reward = apply_to_each_col(reverse_squeeze, board)
    return new_board, reward


@njit
def step_forward(board, action):
    # 0 left, 1 right, 2 up, 3 down
    if action == 0:
        new_board, reward = move_left(board)
    elif action == 1:
        new_board, reward = move_right(board)
    elif action == 2:
        new_board, reward = move_up(board)
    elif action == 3:
        new_board, reward = move_down(board)
    else:
        print("wrong action")
        new_board, reward = None, 0
    return new_board, reward

In [ ]:
def check_row_monotonic(board):
    

In [43]:
a = np.array([[1, 2, 3, 4], [2, 3, 4, 5], [3, 4, 5, 6], [5, 6, 7, 8]])
np.diff(a, axis=1)

array([[1, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [1, 1, 1]])

In [65]:
def monotonic_score(board):
    each_row_diff = np.diff(board, axis=1)
    each_col_diff = np.diff(board, axis=0)
    ascend_row = np.all(each_row_diff >= 0, axis=1) # size: [4]
    descend_row = np.all(each_row_diff <= 0, axis=1)
    
    ascend_col = np.all(each_col_diff >= 0, axis=0)
    descend_col = np.all(each_col_diff <= 0, axis=0)

    interleaving_row_score = max(
        np.sum([ascend_row[0], ascend_row[2], descend_row[1], descend_row[3]]),
        np.sum([descend_row[0], descend_row[2], ascend_row[1], ascend_row[3]])
    )

    interleaving_col_score = max(
        np.sum([ascend_col[0], ascend_col[2], descend_col[1], descend_col[3]]),
        np.sum([descend_col[0], descend_col[2], ascend_col[1], ascend_col[3]])
    )
    return interleaving_row_score, interleaving_col_score


score_base = np.array([
        [2 ** 3,  2 ** 2,  2 ** 1,   2 ** 0],
        [2 ** 4,  2 ** 5,  2 ** 6,   2 ** 7],
        [2 ** 8,  2 ** 9,  2 ** 10,  2 ** 11],
        [2 ** 15, 2 ** 14, 2 ** 13,  2 ** 12]], dtype=np.int32)

def monotonic_score(board):
    all_scores = []
    rt_board = board.copy()
    for i in range(4):
        rt_score = np.sum(np.rot90(rt_board) * score_base)
        tr_score = np.sum(rt_board.T * score_base)
        rt_board = np.rot90(rt_board)
        all_scores.extend([rt_score, tr_score])
    return all_scores
    

In [5]:
# 1
# @njit
def get_max_action_and_rewards(board, search_depth, total_depth):
    if search_depth == total_depth:
        return 0, 0.0, []
    max_action, max_gain = 0, 0.0
    action_gain_pairs = []
    for action in available_actions(board):
        new_board, r1 = step_forward(board, action)
        r2 = get_expected_reward(new_board, search_depth, total_depth)
        r_sum = r1 + r2
        if r_sum > max_gain:
            max_gain = r_sum
            max_action = action
        action_gain_pairs.append((action, r_sum))
    return max_action, max_gain, action_gain_pairs


# @njit
def get_expected_reward(board, search_depth, total_depth):
    # evaluate the random gen
    r_sum = 0
    for (x, y), gen_tile, gen_prob in possible_gen(board, search_depth, total_depth):
        old_value = board[x, y]
        board[x, y] = gen_tile
        action, future_gain, action_gain_pairs = get_max_action_and_rewards(board, search_depth + 1, total_depth)
        board[x, y] = old_value
        r_sum += gen_prob * future_gain
    return r_sum


# @njit
def available_actions(board):
    actions = [0, 1, 2, 3]
    funcs = [move_left, move_right, move_up, move_down]
    available_actions = []
    for action, func in zip(actions, funcs):
        new_board, _ = func(board)
        if np.any(new_board != board):
            available_actions.append(action)
    return available_actions


# @njit
def possible_gen(board, search_depth, total_depth):
    coords = np.argwhere(board == 0)
    if search_depth < 2:
        # get all possible tiles
        gens = [(coord, value, prob * (1 / len(coords))) for coord in coords for value, prob in [(2, 0.9), (4, 0.1)]]
    else:
        # only high possible tiles
        gens = [(coord, value, prob * (1 / len(coords))) for coord in coords for value, prob in [(2, 1)]]
    return gens


In [50]:
board = np.array([
    [1, 2, 2, 0],
    [0, 2, 2, 0],
    [4, 2, 2, 4],
    [4, 4, 4, 4]
])
board = np.array([
    [1, 4, 2, 4],
    [0, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])

In [51]:
%time
get_max_action_and_rewards(board, 0, 4)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.82 µs


(0, 3.48, [(0, 3.48), (2, 0.0), (3, 1.04)])

In [27]:
available_actions(board)

[0, 1]

In [17]:
funcs = [move_left, move_right, move_up, move_down]
funcs[0](board)

(array([[1, 4, 0, 0],
        [4, 0, 0, 0],
        [4, 4, 4, 0],
        [8, 8, 0, 0]]),
 28.0)